# Scraper para o site da Hemeroteca Digital de Lisboa

Esse notebook busca coletar os dados dos periódicos digitalizados e disponíveis no site da Hemeroteca Digital de Lisboa. Os dados serão organizados em um *dataframe* e exportados para um arquivo csv.

Os dados coletados são:

- **Título**: Título do periódico
- **Autoria**: Autoria do periódico
- **Período**: Período disponível do periódico
- **Link**: Link para o periódico

In [1]:
# importar bibliotecas
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
# definir lista de letras que foram o índice
letras = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J','L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U','V', 'X', 'Z']

# definir url base
url_base = 'https://hemerotecadigital.cm-lisboa.pt/Indice/Indice'

# final da url
url_final = '.htm'

In [3]:
# definir lista de urls completa
urls = [url_base + letra + url_final for letra in letras]
print(urls)

['https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceA.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceB.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceC.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceD.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceE.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceF.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceG.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceH.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceI.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceJ.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceL.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceM.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceN.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceO.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceP.htm', 'https://hemerotecadigital.cm-lisboa.pt/Indice/IndiceQ.htm', 'https://hemerotecadigi

In [4]:
# função para criar objeto soup
def criar_soup(url):
    html = requests.get(url)
    soup = bs(html.content, 'html.parser')
    return soup

In [5]:
# find all a href that ends with .htm and start with ../Periodicos or ../OBRAS
def encontrar_links(soup):
    links = soup.find_all('a', href=True)
    links = [link['href'] for link in links if link['href'].endswith('.htm') and (link['href'].startswith('../Periodicos') or link['href'].startswith('../OBRAS'))]
    # replace ../ with https://hemerotecadigital.cm-lisboa.pt/
    links = [link.replace('../', 'https://hemerotecadigital.cm-lisboa.pt/') for link in links]
    return links

In [6]:
# loop com as urls
links_geral = []
for url in urls:
    soup = criar_soup(url)
    links = encontrar_links(soup)
    links_geral.extend(links)

In [7]:
# função para pegar o título e a data de publicação
def pegar_titulo_data(soup):
    titulo_data = soup.find('title').text
    # encontrar data com regex: tudo entre [ e ]
    # se não encontrar [ no título, retorna None
    pattern = r'\['
    if re.search(pattern, titulo_data) == None:
        data = "Sem data"
        titulo = titulo_data
    else:
        data = re.findall(r'\[(.*?)\]', titulo_data)
        # encontrar título com regex: tudo antes de [
        titulo = re.findall(r'(.*?)\[', titulo_data)
    return titulo, data

In [8]:
# Find all link that ends with .pdf
def encontrar_pdf(soup, url):
    pdf_links = soup.find_all('a', href=True)
    pdf_links = [link['href'] for link in pdf_links if link['href'].endswith('.pdf') or link['href'].endswith('.PDF')]
    # excluir string após última "/" em url
    url = url.rsplit('/', 1)[0]
    url = url + '/'
    # add url to pdf_links
    pdf_links = [url + link for link in pdf_links]
    return pdf_links

In [9]:
# criar lista geral para os dados
dados_geral = []
# loop nos links
for link in links_geral:
    soup = criar_soup(link)
    titulo, data = pegar_titulo_data(soup)
    pdf_links = encontrar_pdf(soup, link)
    # se FichasHistoricas estiver no link, adicionar à lista de dados na variável fichas
    fichas = "Sem Ficha Histórica"
    if any("FichasHistoricas" in pdf for pdf in pdf_links):
        fichas = [pdf for pdf in pdf_links if "FichasHistoricas" in pdf]
        # correção do link para o pdf
        fichas = fichas[0].replace(link, 'https://hemerotecadigital.cm-lisboa.pt/')
    # contar o número de pdfs
    num_pdfs = len(pdf_links)
    # verificar se algum pdf possui a string"FichasHistoricas"
    # se sim, excluir da lista e subtrair 1 do número de pdfs
    if any("FichasHistoricas" in pdf for pdf in pdf_links):
        pdf_links = [pdf for pdf in pdf_links if "FichasHistoricas" not in pdf]
        num_pdfs = num_pdfs - 1
    # criar lista com os dados
    dados = [titulo, data, num_pdfs, pdf_links, fichas]
    # adicionar dados à lista de dados
    dados_geral.append(dados)


In [10]:
# criar dataframe com headers
df = pd.DataFrame(dados_geral, columns=['Título', 'Data', 'Quantidade de PDF', 'Links dos PDFs', 'Fichas Históricas'])

In [11]:
# transformar itens da coluna 'Título' em string
df['Título'] = df['Título'].astype(str)
# remover colchetes e aspas da coluna 'Título'
df['Título'] = df['Título'].str.replace('[', '')
df['Título'] = df['Título'].str.replace(']', '')
df['Título'] = df['Título'].str.replace("'", '')
df['Título'] = df['Título'].str.replace('"', '')

/tmp/ipykernel_141236/591487736.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Título'] = df['Título'].str.replace('[', '')
/tmp/ipykernel_141236/591487736.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Título'] = df['Título'].str.replace(']', '')


In [12]:
# transformar itens da coluna 'Data' em string
df['Data'] = df['Data'].astype(str)
# remover colchetes e aspas da coluna 'Data'
df['Data'] = df['Data'].str.replace('[', '')
df['Data'] = df['Data'].str.replace(']', '')
df['Data'] = df['Data'].str.replace("'", '')
df['Data'] = df['Data'].str.replace('"', '')


/tmp/ipykernel_141236/3931564129.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Data'] = df['Data'].str.replace('[', '')
/tmp/ipykernel_141236/3931564129.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Data'] = df['Data'].str.replace(']', '')


In [13]:
# se coluna data for vazia, preencher com "Sem data"
df['Data'] = df['Data'].replace('', 'Sem data')

In [14]:
# se links_pdf não for vazio, pegar o primeiro link
if df['Links dos PDFs'].empty == False:
    # se coluna 'Título' for vazia, preencher com item da coluna 'Links dos PDFs'
    df.loc[df['Título'] == '', 'Título'] = df['Links dos PDFs'].str[0]

In [15]:
# transforma itens da coluna 'Título' em string
df['Título'] = df['Título'].astype(str)


In [16]:
# excluir string até "Periodicos/" ou "OBRAS/" na coluna título usando regex
df['Título'] = df['Título'].str.replace(r'.*Periodicos/', '')
df['Título'] = df['Título'].str.replace(r'.*OBRAS/', '')
# substitui tudo após o primeiro "/" por vazio
df['Título'] = df['Título'].str.replace(r'/.*', '')

/tmp/ipykernel_141236/3298843525.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Título'] = df['Título'].str.replace(r'.*Periodicos/', '')
/tmp/ipykernel_141236/3298843525.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Título'] = df['Título'].str.replace(r'.*OBRAS/', '')
/tmp/ipykernel_141236/3298843525.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Título'] = df['Título'].str.replace(r'/.*', '')


In [17]:
# se valor na coluna 'Título' for repetido, exclui linha
df = df.drop_duplicates(subset=['Título'], keep='first')

In [18]:
df.head(30)

,Título,Data,Quantidade de PDF,Links dos PDFs,Fichas Históricas
0,Academia Portuguesa,1933,11,[https://hemerotecadigital.cm-lisboa.pt/Period...,Sem Ficha Histórica
1,O Académico: publicação bi-semanal redigida po...,Sem data,1,[https://hemerotecadigital.cm-lisboa.pt/Period...,Sem Ficha Histórica
2,O academico: revista quinzenal litteraria,1878,6,[https://hemerotecadigital.cm-lisboa.pt/Period...,https://hemerotecadigital.cm-lisboa.pt/Periodi...
4,O Académico: semanário ilustrado,1902-1903,8,[https://hemerotecadigital.cm-lisboa.pt/Period...,https://hemerotecadigital.cm-lisboa.pt/Periodi...
5,AccaoColonial_NComemorativo,1934,1,[https://hemerotecadigital.cm-lisboa.pt/Period...,Sem Ficha Histórica
6,Acção realista,1924-1926,29,[https://hemerotecadigital.cm-lisboa.pt/Period...,Sem Ficha Histórica
7,"La Acción, Ano 1, n.º 1 (1 de Febrero de 1908)",Sem data,1,[https://hemerotecadigital.cm-lisboa.pt/Period...,Sem Ficha Histórica
8,Actas da Câmara Municipal de Lisboa,1974,3,[https://hemerotecadigital.cm-lisboa.pt/OBRAS/...,Sem Ficha Histórica
9,Actas das sessões da Câmara Municipal de Lisboa,Sem data,8,[https://hemerotecadigital.cm-lisboa.pt/OBRAS/...,Sem Ficha Histórica
11,Água lustral : arte e crítica,"Coimbra, 1913",1,[https://hemerotecadigital.cm-lisboa.pt/Period...,Sem Ficha Histórica


In [19]:
# salvar dataframe em csv
df.to_csv('hemeroteca_lisboa.csv', index=False)